## Homework

I came across my kid's homework which was some kind of simulation
to calculate the probability that a random walk of length 10 didn't leave $[-2,2]$. 

I calculated the value that we should get in the limit  by taking the 10th power of the transition matrix for the Markov process 
- it is 1 - 0.86250402 = 0.13749598
- he told me later they calculated the value using the binomial distribution.

The code was so ugly, an affront to [the Zen of Python](https://www.python.org/dev/peps/pep-0020/),
that I immediately scribbled all over it in red ink.
I don't think a school teacher would care but why do something wrong when it is actually easier to do it right ?

---

### Final solution

This is my final code: 

- 8 LOC as opposed to 18. There is a linebreak in the call to np.random.randint because I hate code that goes too far to the right.
- I used **numpy** to vectorize and this makes things harder to follow but you can't knock a 40x speed gain 
- there is a mathematical trick in L12 to calculate a running tally using an upper triangular matrix. Tricks are **bad** but that's vectorisation for you.

The code is readable for a mathematician used to matrix operations but for a bog standard programmer it **will probably** be much harder. I would argue that now with the developments in machine learning that inherently require vectorisation the programmer should buck up.

Actually I ended up doing about the same speedwise with 
[Cython](http://cython.org/) but that's life. Perhaps the Cython version is better as it is really, really easy to read for a beginner programmer though there is a trick - recoding **random.randint** using **rand** from [libc.sdlib](https://en.wikipedia.org/wiki/C_standard_library).



In [709]:
%%time
import numpy as np

batch_size = 10**6
walk_length = 10
#make a random matrix and with values in -1,0,1
path_deltas = np.random.randint(-1, high=2, 
                                size=(batch_size, walk_length))

#make an upper triangular matrix and calculate postions
UT = np.triu(np.ones((10,10)))
positions = np.dot(path_deltas,UT)

#calculate fancy indexing and use fancy indexing to finish
max_dist_to_0 = np.abs(positions).max(axis=1)
print 'total', len(max_dist_to_0[ max_dist_to_0 < 2])

#dists_to_0 = np.abs(positions[-1,:])
#print 'dist', np.sum(dists_to_0)/batch_size

total 137987
CPU times: user 388 ms, sys: 74.8 ms, total: 463 ms
Wall time: 389 ms


## Original code

This was the original code in the homework.
Actually it didn't respect spacing conventions 
but that's another story.

It's 18 LOC if we don't count the first 3.

It looks all wrong: 
- a **while** where the maximal number of loops is known
- the mere fact that 
it sets a variable **t**  instead of using
multiple **returns**...

In [412]:
%%time
from random import randint

def une_marche():
    x = 0
    y = 0
    while  -1 <= x and x <= 1 and y < 10:
        a = randint(-1,1)
        x = x + a
        y = y + 1
        
    if x == -2 or x == 2:
        t = 0
    else:
        t = 1     
    return t
    
T = 0
num_tries = 10**5
for k in xrange(num_tries):
    T = T + une_marche()
print float(T)/num_tries  

0.13776
CPU times: user 1.81 s, sys: 17.9 ms, total: 1.83 s
Wall time: 1.83 s


###  My first version

- tidied up the while loop
- use and in statement instead of repeated comparisons
- the += operator here and there

For me this still looks like someone that does Java/C++.

In [415]:
%%time
from random import randint

def une_marche():
    x = y = 0
    while  -1 <= x <= 1 and y < 10:
        a = randint(-1,1)
        x += a
        y += 1
    if x in [-2,2]:
        return 0
    else:
        return 1
    
T = 0
num_tries = 10**5
for k in xrange(num_tries):
    T += une_marche()
print float(T)/num_tries    

0.13597
CPU times: user 1.9 s, sys: 18.1 ms, total: 1.91 s
Wall time: 1.93 s


## Second version

A somewhat simpler and uses standard Python idioms.

- for  statement
- list comprehension [ x for x in list]
- the += operator
- the sum builtin 
- the break condition is now abs(x) > 1

It's 10 LOC - so nearly 40% less lines less.

It runs about as fast as my first attempt 
but was marginally faster 
- with -1 <= x <= 1 
- instead of abs(x) > 1

In [471]:
%%time
def une_marche2():
    x = 0
    for y in range(10): 
        x += randint(-1,1)
        if abs(x) > 1 : return 0
    return 1

num_tries = 10**5
tries = [une_marche2() for k in xrange(num_tries)]
print float(sum(tries))/num_tries

0.13741
CPU times: user 1.7 s, sys: 24.8 ms, total: 1.72 s
Wall time: 1.72 s


An alternative version that uses
- for break else  statement

In [527]:
%%time
def une_marche2():
    x = 0
    for y in range(10): 
        x += randint(-1,1)
        if abs(x) > 1 : break
    else:
        return 1
    return 0

num_tries = 10**6
tries = [une_marche2() for k in xrange(num_tries)]
print float(sum(tries))/num_tries

0.137023
CPU times: user 17 s, sys: 122 ms, total: 17.1 s
Wall time: 17.2 s


## Cython

Well this code is so cute why not try and compile it.
The speed seems to be bound by a call to **randint** :(

See:
https://github.com/kwmsmith/scipy-2017-cython-tutorial/blob/master/02-cython-comparison.ipynb


See also:
https://www.ibm.com/developerworks/community/blogs/jfp/entry/Python_Meets_Julia_Micro_Performance?lang=en

and maybe we need more work :(
http://nealhughes.net/cython1/


In [482]:
%load_ext Cython

In [714]:
%%cython 
from random import randint

def une_marche_cy():
    cdef int x = 0
    cdef int y
    for y in range(10): 
        x +=  randint(-1,1)
        if abs(x) > 1 : return <int> 0
    return <int> 1
    
def run_cy1(long num_tries = 10**5): 
    tries = [une_marche_cy() for k in xrange(num_tries)]
    return tries


In [716]:
%%time
tries = run_cy1(num_tries = 10**6)
print float(sum(tries))/len(tries)

0.13789
CPU times: user 13.1 s, sys: 84.6 ms, total: 13.2 s
Wall time: 13.2 s


### Conclusion 1

- times to about 13.1 for a batch of $10^6$ which is nearly  4secs faster than the pure Python version but way slower than **numpy**.
- the bottleneck is the call to **randint**


So I found this as a remedy to the problem with the call.

http://hplgit.github.io/teamods/MC_cython/main_MC_cython.html

Actually there is an amusing mistake in their code:

r = 1 + int(rand()/(RAND_MAX*6.0))

The parenthesis are wrong here (RAND_MAX*6.0)
what they want is 

r = 1 + int(6.0*rand()/RAND_MAX)

In [738]:
%%cython 
from libc.stdlib cimport rand, RAND_MAX

def my_rand():
    '''integer in -1,0,1'''
    return -1 + int(3.0*rand()/RAND_MAX) 

def une_marche_cy():
    cdef int x = 0, y
    for y in range(10): 
        x +=  my_rand()
        if abs(x) > 1 : return  0
    return  1

### Conclusion 2

Still very cute for all I have done is 
1. add a type statement **cdef**
2. written a replacement for **random.randint(-1,1)**.

Very satisfying n'est-ce pas ?
I could inline the code for **my_rand** to avoid a function call
and

- squeeze a few microseconds.
- get down to 10 LOC

but who cares?

And the moment of truth...

In [741]:
%%time
num_tries = 10**6
tries = [une_marche_cy() for k in xrange(num_tries)]
print float(sum(tries))/len(tries)

0.137002
CPU times: user 623 ms, sys: 30.4 ms, total: 654 ms
Wall time: 691 ms


slower than the **numpy** version but that's only because
of the last cell doing some calculations in pure Python :).

## The numpy  version

This is basically about vectorisation and matrix multiplication using **numpy**.

Just dotting with a vector of ones doesn't work
because the walk can break out of $[-1,1]$ and get back in again.
So we use an upper triangular matrix to calculate a running total 
of the position deltas.

- I don't really care about memory usage
- this is definitely not for les tapettes
- finally optimisation of counting solutions is tricky

### Optimisation

Finally I found this which is kind of obvious as an afterthought
https://stackoverflow.com/questions/36267936/normalizing-rows-of-a-matrix-python


CPU times: user 411 ms, sys: 84.4 ms, total: 495 ms

This compares very favorably with my first version
running on 10**6 :

CPU times: user 17.6 s

In [321]:
%%time
import numpy as np

n = 10**6
M = np.random.randint(3, size=(n,10 ))
M -= np.ones_like(M)

Y = np.triu(np.ones((10,10)))
walks = np.dot(M,Y)

#this is a bit fecked but I don't see how 
#to easily vectorize this
#CPU times: user 3.71 s, sys: 229 ms, total: 3.94 s
#bounds = np.array([max(abs(x)) for x in walks])
#len(bounds[bounds < 2])

#this didn't work - twise as long on 10**6
#CPU times: user 440 ms, sys: 86.5 ms, total: 527 ms
lengths = np.abs(walks).max(axis=1)
print len(lengths[lengths < 2])

137831
CPU times: user 411 ms, sys: 84.4 ms, total: 495 ms
Wall time: 384 ms


## Now let's get the value using a Markoff process

calulate the exact value using a Markoff process on the graph

In [404]:
A = np.array([[1,1,0,1], [1,1,1,0],[0,1,1,1],[0,0,0,3]])
A = A.transpose()/3.
A

array([[ 0.33333333,  0.33333333,  0.        ,  0.        ],
       [ 0.33333333,  0.33333333,  0.33333333,  0.        ],
       [ 0.        ,  0.33333333,  0.33333333,  0.        ],
       [ 0.33333333,  0.        ,  0.33333333,  1.        ]])

In [370]:
V = np.array([0,1,0,0]).reshape((-1,1))

In [386]:
sp, vp = np.linalg.eig(A)

In [393]:
vv = vp[:,0].reshape((-1,1))

In [394]:
np.dot(A,vv)

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 1.]])

In [402]:
np.linalg.matrix_power(A,10)

array([[ 0.02848482,  0.04027164,  0.02846788,  0.        ],
       [ 0.04027164,  0.0569527 ,  0.04027164,  0.        ],
       [ 0.02846788,  0.04027164,  0.02848482,  0.        ],
       [ 0.90277566,  0.86250402,  0.90277566,  1.        ]])

In [405]:
1 - 0.86250402


0.13749597999999996

In [435]:
np.random.randint(-1,high=2,size=(1,10))

array([[ 1,  0,  0, -1, -1,  0,  0,  1, -1,  0]])

## Appendix

So I found [this](http://hplgit.github.io/teamods/MC_cython/main_MC_cython.html)
which says that you should pull stuff from **libc**. Frankly I don't reall know what's there...

There is also [this replacement for random](https://github.com/Noctem/cyrandom).


The CPU time of the plain Python version was 10 s, which is reasonably fast for obtaining a fairly accurate result in this problem. The lesson learned is therefore that a Monte Carlo simulation can be implemented in plain Python first. If more speed is needed, one can just add type information and create a Cython code. Studying the HTML file with what Cython manages to translate to C may give hints about how successful the Cython code is and point to optimizations, like avoiding the call to random.randint in the present case. Optimal Cython code runs here at approximately the same speed as calling a handwritten C function with the time-consuming loops. It is to be noticed that the stand-alone C program here ran faster than calling C from Python, probably because the amount of calculations is not large enough to make the overhead of calling C negligible.

Vectorized Python do give a great speed-up compared to plain loops in Python, if done correctly, but the efficiency is not on par with Cython or handwritten C. Even more important is the fact that vectorized code is not at all as readable as the algorithm expressed in plain Python, Cython, or C. Cython therefore provides a very attractive combination of readability, ease of programming, and high speed.



In [662]:
%%cython 
from libc.stdlib cimport rand, RAND_MAX

def rr():
    return int( 3.0*rand()/RAND_MAX ) -1


In [692]:
pp = [ sum([rr() for i in range(30)]) for k in range(1000)]
float(sum(pp))/len(pp)

0.007